# Imports & Dijkstra Algorithm

In [ ]:
import numpy as np
import os
from itertools import combinations
import random
import sys
import pandas as pd

In [ ]:
# Class and Functions for Dijkstra-Algorithm

class Graph(object):
    def __init__(self, nodes, init_graph):
        self.__nodes = nodes
        self.__graph = self.__construct_graph(nodes, init_graph)

    @staticmethod
    def __construct_graph(nodes, init_graph):
        """
        This method makes sure that the graph is symmetrical. In other words, if there's a path from node A to B with a value V,
        there needs to be a path from node B to node A with a value V.
        """
        graph = {}
        for node in nodes:
            graph[node] = {}

        graph.update(init_graph)

        for node, edges in graph.items():
            for adjacent_node, value in edges.items():
                if not graph[adjacent_node].get(node, False):
                    graph[adjacent_node][node] = value

        return graph

    def get_nodes(self):
        """Returns the nodes of the graph."""
        return self.__nodes

    def get_outgoing_edges(self, node):
        """Returns the neighbors of a node."""
        connections = []
        for out_node in self.__nodes:
            if self.__graph[node].get(out_node, False):
                connections.append(out_node)
        return connections

    def value(self, node1, node2):
        """Returns the value of an edge between two nodes."""
        return self.__graph[node1][node2]

def dijkstra_algorithm(graph: Graph, start_node: str):
    """
    Calculates shortest paths with costs in passed graph to each other nodes from the passed start node
    """
    unvisited_nodes = list(graph.get_nodes())

    # We'll use this dict to save the cost of visiting each node and update it as we move along the graph
    shortest_path = {}

    # We'll use this dict to save the shortest known path to a node found so far
    previous_nodes = {}

    # We'll use max_value to initialize the "infinity" value of the unvisited nodes
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # However, we initialize the starting node's value with 0
    shortest_path[start_node] = 0

    # The algorithm executes until we visit all nodes
    while unvisited_nodes:
        # The code block below finds the node with the lowest score
        current_min_node = None
        for node in unvisited_nodes: # Iterate over the nodes
            if current_min_node is None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node

        # The code block below retrieves the current node's neighbors and updates their distances
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # We also update the best path to the current node
                previous_nodes[neighbor] = current_min_node

        # After visiting its neighbors, we mark the node as "visited"
        unvisited_nodes.remove(current_min_node)

    return previous_nodes, shortest_path

# For manual testing during development
def print_result(previous_nodes, shortest_path, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]

    # Add the start node manually
    path.append(start_node)

    print("Test Output:")
    print("We found the following best path with a value of {}.".format(shortest_path[target_node]))
    print(" -> ".join(reversed(path)))
    
def get_route(previous_nodes, shortest_path, start_node, target_node):
    path = []
    node = target_node

    while node != start_node:
        path.append(node)
        node = previous_nodes[node]

    # Add the start node manually
    path.append(start_node)
    return path

# Configuration

For each entity stations, trains, lines and passengers a dictionary can be defined that determines the attributes of the instances.

## stations configuration

- numberOfStations: defines the number of stations in the input-file
- stationsMaxTrainCapacity: maximal capacity of trains in a station. A station gets a random capacity of trains between 1 and that value

In [ ]:
stationsConfig = {
    'numberOfStations': 215,
    'stationsMaxTrainCapacity': 10 
}

## trains configuration

- numberOfTrains: defines the number of trains in the input-file
- minSpeed: defines the minimal velocity of a train
- maxSpeed: defines the maximal velocity of a train
- minCapacityOfPassengers: defines the minimal number of passengers that can board on a train
- maxCapacityOfPassengers: defines the maximal number of passengers that can board on a train
- numberOfTrainsWith*: defines the number of trains with an arbitrary starting position

In [ ]:
trainsConfig = {
    'numberOfTrains': 10,
    'maxSpeed': 12,
    'minSpeed': 4,
    'minCapacityOfPassengers': 20,
    'maxCapacityOfPassengers': 30,
    'numberOfTrainsWith*': 8
}

## lines configuration

- numberOfLines: numberOfLines in the input-file. Has to be smaller than the maximal possible value (see below)
- minLength: minimal possible length of a line
- maxLength: maximal possible length of a line
- maxCapacity: maximal number of trains on a line

In [ ]:
#maximal possible number of lines when every station is directly connected to every other station:
print('Maximal possible number of lines is:', np.sum([i for i in range(stationsConfig['numberOfStations'])]))

In [ ]:
linesConfig = {
    'numberOfLines': 18426,
    'minLength': 5,
    'maxLength': 10,
    'maxCapacity': 4
}

if linesConfig['numberOfLines'] > np.sum([i for i in range(stationsConfig['numberOfStations'])]):
    raise Exception('numberOfLines is larger than the maximal possible number of lines given the current number of stations.')

## passengers configuration

- numberOfPassengers: defines the number of passengers in the input-file
- maxGroupsize: defines the maximal size of a group of passengers. MUST NOT BE LARGER THAN THE MAX TRAINS CAPACITY!
- arrivalTime:
 - minimal: minimal cost from start station to end station without considering train speed
 - minimal_with_train_speed: minimal cost from start station to end station with considerung train speed. Takes the fastest train in the start station. If there is on train in the start station the fastest train with * as start station will be used

In [ ]:
print('Largest possible groupsize is', trainsConfig['maxCapacityOfPassengers'])

In [ ]:
passengersConfig = {
    'numberOfPassengers': 721,
    'maxGroupsize': 20,
    #'arrivalTime': 'minimal',
    #'arrivalTime': 'random',
    'arrivalTime': 'minimal_with_train_speed',
}

## general configuration

In [ ]:
generalConfig = {
    'includeConfigString': False,
    #'filename': 'custom_large_input.txt',
}

# Create data

## create stations

In [ ]:
# building the string according to the standardized input-format
stationsString = '#Stations: ID, capacity\n'
stationsString += '[Stations]\n'

# creating the station-IDs and selecting a random capacity for each station 
for i in range(stationsConfig['numberOfStations']):
    singleStationString = 'S' + str(i + 1) + ' ' + str(np.random.randint(stationsConfig['stationsMaxTrainCapacity']) + 1) + '\n'
    stationsString += singleStationString

stationsString += '\n' 

In [ ]:
#print(stationsString)

## create trains

In [ ]:
def get_all_stations():
    """
    returns a list of the IDs of all stations
    """
    stations = stationsString.split('\n')
    stations.remove('[Stations]')
    while '' in stations: 
        stations.remove('')
    stations.remove('#Stations: ID, capacity')

    for elem in stations:
        station = elem.split(' ')
        stations[stations.index(elem)] = station

    return stations

In [ ]:
# building the string according to the standardized input-format
trainsString = '#Trains: ID, start station, velocity, capacity\n'
trainsString += '[Trains]\n'

# getting all stations, used to select a random starting and end station for each train out of this pool
stations = get_all_stations()
stations = pd.DataFrame(stations, columns=['name', 'capacity'])
stations.set_index('name', inplace=True)
stations = stations.astype('int32')
stations['trains_in_station'] = np.zeros(stations.size, dtype=int)

for i in range(trainsConfig['numberOfTrains']):
    # select starting station for each train
    if i < trainsConfig['numberOfTrainsWith*']:
        # assign an arbitrary starting station ('*') to the current train
        startingStation = '*'
    else:
        # find a station that can accomodate the current train
        find_station = True
        while find_station:
            temp_starting_station = 'S' + str(np.random.randint(stationsConfig['numberOfStations']) + 1)
            if stations.loc[temp_starting_station]['trains_in_station'] < stations.loc[temp_starting_station]['capacity']:
                startingStation = temp_starting_station
                stations.loc[startingStation]['trains_in_station'] += 1
                find_station = False

    # select speed of train
    speed = str(round(random.uniform(trainsConfig['minSpeed'], trainsConfig['maxSpeed']), 5))

    # select capacity of passengers in train
    capacity = str(np.random.randint(trainsConfig['minCapacityOfPassengers'], trainsConfig['maxCapacityOfPassengers'] + 1))
    
    # select the train id and combine all selected attributes in one string
    singleTrainString = 'T' + str(i + 1) + ' ' + startingStation + ' ' + speed + ' ' +  capacity + '\n'
    trainsString += singleTrainString

trainsString += '\n'

In [ ]:
#print(trainsString)

## create lines

In [ ]:
def createAllLines():
    """
    Creates all possible lines so that every station is directly connected by a line to every other station.
    Randomizes the order of the created lines and saves them in the list allPairsList.
    """
    allStations = ['S' + str(i+1) for i in range(stationsConfig['numberOfStations'])]
    allPairs = combinations(allStations, 2)
    allPairsList = []
    for pair in allPairs: 
        allPairsList.append(list(pair))

    #randomize the order of the lines:
    random.shuffle(allPairsList)
    return allPairsList

In [ ]:
def removeLines(printOrder=False):
    """
    Iterate over every line in allPairsList and check whether the removal of that line would result in a disconnected station.
    Repeat until the number of lines left is equal to the numberOfLines in linesConfig or until we can not remove another line 
    without disconnecting the graph. 
    """
    allStations = ['S' + str(i+1) for i in range(stationsConfig['numberOfStations'])]
    allPairsList = createAllLines()
    lines = allPairsList.copy()

    for pair in allPairsList:
        #check if the number of lines is larger than numberOfLines from linesConfig. If that is not the case stop the algorithm.
        if len(lines) <= linesConfig['numberOfLines']:
            if printOrder: 
                print('numberOfLines reached, no more lines will be removed.')
            break

        tempList = lines.copy()

        #create graph with all stations
        init_graph = {}
        for station in allStations:
            init_graph[station] = {}

        #remove a line
        tempList.remove(pair)

        #add all lines except the lines that were removed in the previous and in the current step
        for line in tempList:
            #initialize all lines with weight 1 (weight is irrelevant for this task)
            init_graph[line[0]][line[1]] = 1

        graph = Graph(allStations, init_graph)

        #check wheter the removal of the line results in a disconnected station
        #example: if a line that connects S1 and S2 is removed and results in a disconnected station either S1 or S2 must be
        #         disconnected. Thus it is sufficient to check whether S1 or S2 are disconnected.
        previousNodesOfFirstStation, shortestPathOfFirstStation = dijkstra_algorithm(graph=graph, start_node=pair[0])
        previousNodesOfSecondStation, shortestPathOfSecondStation = dijkstra_algorithm(graph=graph, start_node=pair[1])

        #in the Dijkstra algorithm disconnected stations are marked with a distance of sys.maxsize
        if sys.maxsize in shortestPathOfFirstStation.values() or sys.maxsize in shortestPathOfSecondStation.values():
            #if the removal of the line would result in a disconnected station simply move on without action
            if printOrder:
                print('The removal of ', pair, 'would result in a disconnected station.')
            continue
        else:
            #remove the line if it does not disconnect a station
            if printOrder:
                print('The line ', pair, 'can be removed.')
            lines.remove(pair)

    #sort lines
    lines.sort()

    if len(lines) > linesConfig['numberOfLines']:
        if printOrder:
            print('numberOfLines (' + str(linesConfig['numberOfLines'],) + ') from configuration could not be reached. Seemingly the minimum number of lines is ' +str(len(lines)) + '.')
    return lines

In [ ]:
# build string according to standardized input-format
linesString = '#Lines: ID, first station, second station, length, capacity\n'
linesString += '[Lines]\n'
lines = removeLines()

for i in range(len(lines)):
    # select length of line according to configurations
    length = round(random.uniform(linesConfig['minLength'], linesConfig['maxLength']), 5)

    # select capacity of line according to configurations
    capacity = np.random.randint(1, linesConfig['maxCapacity'] + 1)
    
    # combine line ID and the other attributes in one string
    singleLineString = 'L' + str(i + 1) + ' ' + str(lines[i][0]) + ' ' + str(lines[i][1]) + ' ' + str(length) + ' ' + str(capacity) + '\n'
    linesString += singleLineString

linesString += '\n'

In [ ]:
#print(linesString)

## create passengers

In [ ]:
#create graph to determine minimal arrival time
allStations = ['S' + str(i+1) for i in range(stationsConfig['numberOfStations'])]
init_graph = {}
for station in allStations:
    init_graph[station] = {}

#get all lines
lines = linesString.split('\n')
lines.remove('[Lines]')
while '' in lines: 
    lines.remove('')
lines.remove('#Lines: ID, first station, second station, length, capacity')

#initialize all lines with corresponding start station, end station and length
for elem in lines:
    line = elem.split(' ')
    lines[lines.index(elem)] = line
    firstStation = line[1]
    secondStation = line[2]
    length = float(line[3])
    init_graph[firstStation][secondStation] = length

#create graph
graph = Graph(allStations, init_graph)

In [ ]:
def get_all_trains():
    """
    returns a list of the IDs of all trains
    """
    trains = trainsString.split('\n')
    trains.remove('[Trains]')
    while '' in trains: 
        trains.remove('')
    trains.remove('#Trains: ID, start station, velocity, capacity')

    for elem in trains:
        train = elem.split(' ')
        trains[trains.index(elem)] = train

    return trains

def get_fastest_train_in_station(start_station):
    """
    Returns the fastest available train for start_station.
    If there is no train in start_station the fastest train with an arbitrary starting station is returned.
    """
    
    #get all trains
    trains = get_all_trains()
    
    #get all trains in start_station
    trains_in_station = []
    for train in trains:
        if start_station in train:
            trains_in_station.append(train)
        
    #get fastest train in start station
    for train in trains_in_station:
        if train[2] == max([temp_train[2] for temp_train in trains_in_station]):
            return train
    
    #if there is no train in station: get fastest train with * as start station
    arbitrary_trains = [train for train in trains if '*' in train]
    for arbitrary_train in arbitrary_trains:
        if arbitrary_train[2] == max([train[2] for train in arbitrary_trains]):
            return arbitrary_train

In [ ]:
# build string according to standardized input-format
passengersString = '#Passengers: ID, start station, end station, groupsize, expected arrival time\n'
passengersString += '[Passengers]\n'

for i in range(passengersConfig['numberOfPassengers']):
    # select random start and end station
    tempStations = allStations.copy()
    startStation = allStations[np.random.randint(len(allStations))]
    tempStations.remove(startStation)
    endStation = tempStations[np.random.randint(len(tempStations))]
    
    # select random group size
    groupSize = str(np.random.randint(1, passengersConfig['maxGroupsize'] + 1))
    
    # select arrival time
    if passengersConfig['arrivalTime'] == 'minimal':
        # set desired arrival time to total length of route
        previousNodes, shortestPath = dijkstra_algorithm(graph=graph, start_node=startStation)
        arrivalTime = str(shortestPath[endStation])
        
    if passengersConfig['arrivalTime'] == 'minimal_with_train_speed':
        # set desired arrival time for a passenger p with starting station s and destination d 
        # to the time it would take the fastest available train to get p from s to d 
        previousNodes, shortestPath = dijkstra_algorithm(graph=graph, start_node=startStation)
        # get cost of lines along route to endStation
        route = get_route(previousNodes, shortestPath, startStation, endStation)   #get the route to the passengers endStation
        lines_on_route = [[route[i], route[i+1]] for i in range(len(route)-1)]     #get all lines along the route
        costs_on_route = []                                                        #get the cost for each line without considering train speed
        for line_on_route in lines_on_route:
            for line in lines:
                if line_on_route[0] in line and line_on_route[1] in line:
                    costs_on_route.append(line[3])
                    
        # get fastest train in current station
        fastest_train = get_fastest_train_in_station(startStation)
        speed = fastest_train[2]
        
        # get costs in rounds considering train speed
        costs = [int(float(cost)/float(speed))+1 for cost in costs_on_route]
        arrivalTime = str(sum(costs))
    
    # build string for the current passenger
    singlePassengerString = 'P' + str(i + 1) + ' ' + startStation + ' ' + endStation + ' ' + groupSize + ' ' + arrivalTime + '\n'
    passengersString += singlePassengerString
    
passengersString += '\n' 

# Create output file

In [ ]:
def createConfigString():
    configString = ''

    if generalConfig['includeConfigString'] == True:
        configString = '#Configurations: \n'

        configString += '\n#General configurations:\n'
        for key, value in generalConfig.items():
            configString += '#' + key + ': ' + str(value) + '\n'

        configString += '\n#Stations configurations:\n'
        for key, value in stationsConfig.items():
            configString += '#' + key + ': ' + str(value) + '\n'

        configString += '\n#Trains configurations:\n'
        for key, value in trainsConfig.items():
            configString += '#' + key + ': ' + str(value) + '\n'

        configString += '\n#Lines configurations:\n'
        for key, value in linesConfig.items():
            configString += '#' + key + ': ' + str(value) + '\n'
            
        configString += '\n#Passengers configurations:\n'
        for key, value in passengersConfig.items():
            configString += '#' + key + ': ' + str(value) + '\n'

        configString += '\n'
        return configString

In [ ]:
#print(configString)

## Create file

In [ ]:
# include the configuration as comments if this is specified in the config-dictionary
if generalConfig['includeConfigString']:
    outputString = createConfigString() + stationsString + linesString + trainsString + passengersString
else:
    outputString = stationsString + linesString + trainsString + passengersString

In [ ]:
try:
    print(generalConfig['filename']) # Throws an error if not set

    with open(generalConfig['filename'], 'w') as f:
        f.write(outputString)
except:
    # Automatically create filename from settings in the config-dictionaries
    filenames = (
        file for file in os.listdir('../Input')
            if os.path.isfile(os.path.join('../Input', file))
    )
    # filename consists of the number of elements plus the specified attributes for each entity
    generated_filename = str(stationsConfig['numberOfStations']) \
                         + '_' + str(stationsConfig['stationsMaxTrainCapacity']) \
                         + '-' + str(trainsConfig['numberOfTrains']) \
                         + '_' + str(trainsConfig['maxSpeed']) \
                         + '_' + str(trainsConfig['minCapacityOfPassengers']) \
                         + '_' + str(trainsConfig['maxCapacityOfPassengers']) \
                         + '_' + str(trainsConfig['numberOfTrainsWith*']) \
                         + '-' + str(linesConfig['numberOfLines']) \
                         + '_' + str(linesConfig['minLength']) \
                         + '_' + str(linesConfig['maxLength']) \
                         + '_' + str(linesConfig['maxCapacity']) \
                         + '-' + str(passengersConfig['numberOfPassengers']) \
                         + '_' + str(passengersConfig['maxGroupsize']) \
                         + '_input.txt'
    
    # add prefix to display selection configuration of passengers desired arrival time
    if passengersConfig['arrivalTime'] == 'minimal_with_train_speed':
        generated_filename = 'custom_min_' + generated_filename
    elif passengersConfig['arrivalTime'] == 'minimal':
        generated_filename = 'custom_min_' + generated_filename
    else:
        generated_filename = 'custom_' + generated_filename

    with open(generated_filename, 'w') as f:
        f.write(outputString)

    print('\n-------------- DONE --------------')